In [1]:
import os
import numpy as np
import pandas as pd

In [115]:
# Load dataset.

################################################################################################
###  Filename is based on Preprocessing notebook, please check there if you get any errors   ###
################################################################################################

dfpath = os.path.join(os.getcwd(), "data", "filtered_long.csv")
print("Loading file from: {}".format(dfpath))

df = pd.read_csv(dfpath, header=0)
df.head()

Loading file from: D:\GroupAssignment\data\filtered_long.csv


,Unnamed: 0,session,purchase,len,add
0,0,"[1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, ...",0,18,2
1,1,"[1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, ...",0,139,18
2,2,"[1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ...",0,41,16
3,3,"[1, 2, 3, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,16,2
4,4,"[1, 1, 1, 1, 2, 3, 4, 1, 2, 1, 1, 1, 4, 4, 4, ...",1,29,5


In [117]:
# Format the sessions from a string in the form [1,2,3] to a working tuple
df.session = df.session.map(lambda x: tuple(map(int, x.strip("[]").split(","))))
type(df.session[0])

tuple

In [118]:
# Split set into training and validation set
from sklearn.model_selection import train_test_split

# No real reason for the selected ratio. In PS they used 70/30. 

######################################
###   Don't forget to set state.   ###
######################################

training_set, eval_set = train_test_split(df, test_size=0.2, random_state=123, shuffle=False)
eval_set.head()

,Unnamed: 0,session,purchase,len,add
129080,129080,"(1, 1, 2, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, ...",0,17,6
129081,129081,"(1, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 3, 1)",0,16,14
129082,129082,"(1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, ...",0,38,19
129083,129083,"(1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1, 1)",1,13,5
129084,129084,"(1, 1, 2, 3, 1, 1, 2, 3, 1, 1, 2, 1, 1, 1, 1, ...",1,20,3


In [233]:
from sklearn import metrics
from random import random, seed

# 
seed(123)

def fit_oracle(training, max_clicks):
    
    ### Max clicks determines: 
    ### - Which sessions are evaluated. Any sessions with a "tail" shorter than max clicks are dropped
    ### - Where sessions are cut. A session with a "tail" longer than max clicks is sliced
    training = training[(training['len'] - training['add']) > max_clicks]
    training.session = training.apply(lambda x: x.session[:(x['add'] + max_clicks + 1)], axis=1)

    ### Bit wasteful (we now have three partial dataframes and a dict) but easiest way
    ### to get the buy / no buy frequencies of each pattern in the training set
    buy   = training.session[training['purchase'] == 1].value_counts()
    nobuy = training.session[training['purchase'] == 0].value_counts()    
    
    oracle = dict.fromkeys(training.session)
    
    ### Calculate ratio of sessions that end in a purchase for each session in the dataset
    for key in oracle.keys():
        nb, nn = buy.get(key, default = 0), nobuy.get(key, default = 0)
        oracle[key] = nb / (nb + nn)
    
    return oracle

def predict_oracle(oracle, test, max_clicks):
    
    ### Same process as on the training set. Could put this in a separate function
    test = test[(test['len'] - test['add']) > max_clicks]
    test.session = test.apply(lambda x: x.session[:(x['add'] + max_clicks) + 1], axis=1)    

    ### There are several methods to deal with missing patterns (which is most of them)
    ### Just make sure to comment out all unused options
    
    # Option 1: Assign all missing patterns as no-purchase
    #predict = test.session.map(lambda x: 1 if oracle.get(x, 0) > 0.5 else 0)

    # Option 2: Random guess missing patterns
    predict = test.session.map(lambda x: 1 if oracle.get(x, random()) > 0.5 else 0)
    
    # Uncomment in case of options 1, 2:
    true = test.purchase
    
    # Option 3: Ignore missing patterns entirely (uses a trivial amount of the test set)
    """
    predict = []
    true = []
    for i, k in test.iterrows():
        if oracle.get(k.session) != None:
            if oracle.get(k.session) > 0.5:
                predict.append(1)
            else:
                predict.append(0)
            true.append(k['purchase'])
    """
    f1 = metrics.f1_score(predict, true)
    return f1

In [234]:
oracle = fit_oracle(training_set, 5)
p5 = predict_oracle(oracle, eval_set, 5)
print(p5)


F:\Anaconda\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0.2569464614193049
